### Missing data

- Missing values is a common scenario in a real dataset. 
- Also missing data can be represent in dataset in different ways. 
- It could be like NA or NULL or n.a or ? or - .
- We need to ensure all such reprentation of missing data are treated as missing in analysis
- Invalid values, like negative salary , negative age, etc can also to be treated as missing value

In [1]:
import numpy as np
import pandas as pd

In [2]:
emp = pd.read_csv(r'./data/employees_data.csv')

In [3]:
emp.shape

(1000, 6)

In [4]:
emp.head()

,First Name,Gender,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,97308,6.945,TRUE,Marketing
1,Thomas,Male,61933,NaN,TRUE,NaN
2,Maria,Female,130590,11.858,FALSE,Finance
3,Jerry,Male,NaN,9.34,TRUE,Finance
4,Larry,Male,101004,1.389,TRUE,Client Services


In [34]:
# rename columns to replace space with underscore, remove special symbols like %
emp.columns = emp.columns.str.replace(r'\s','_').str.replace(r'\W','').str.replace(r'_$','')

In [47]:
emp.head(n = 3)

,First_Name,Gender,Salary,Bonus,Senior_Management,Team
0,Douglas,Male,97308,6.945,TRUE,Marketing
1,Thomas,Male,61933,NaN,TRUE,NaN
2,Maria,Female,130590,11.858,FALSE,Finance


In [35]:
# Lets start with Gender column
emp.Gender.unique()

array(['Male', 'Female', 'n.a.', nan], dtype=object)

'n.a.' should be treated as missing value

In [8]:
sum(emp.Gender == 'n.a.')

1

In [9]:
emp.Gender.isna().sum()

148

In [10]:
# Salary

emp.Salary.isna().sum()

2

In [13]:
emp.Salary.dtypes

dtype('O')

In [37]:
sal = emp.Salary.copy()

def as_integer(x):
    try:
        return int(x)
    except:
        return np.nan
    
sal = sal.apply(as_integer)    

In [38]:
emp.Salary[sal.isna()]

3       NaN
8       NaN
319    n.a.
349       ?
427       ?
Name: Salary, dtype: object

So n.a. and ? are invalid values in Salary

In [46]:
sum(sal <= 0)

0

In [40]:
# Bonus
emp.Bonus.dtypes

dtype('O')

In [41]:
emp.Bonus.isna().sum()

3

In [43]:
bonus = emp.Bonus.copy()

def as_float(x):
    try:
        return float(x)
    except:
        return np.nan

bonus = bonus.apply(as_float)

emp.Bonus[bonus.isna()]

1     NaN
14    NaN
47    NaN
75      ?
Name: Bonus, dtype: object

? is invalid value in Bonus

In [48]:
# Senior_Management	
emp.Senior_Management.unique()

array(['TRUE', 'FALSE', nan, 'n.a.', '?'], dtype=object)

'n.a.', '?' are invaid values

In [49]:
# Team 
emp.Team.unique()

array(['Marketing', nan, 'Finance', 'Client Services', 'Legal', 'Product',
       'Engineering', 'Business Development', 'Human Resources', 'Sales',
       'na', 'Distribution', 'n.a.', '?'], dtype=object)

'na', 'n.a.', '?' should be marked as missing values

In [52]:
# Name
emp.First_Name.isna().sum()

69

In [56]:
emp.dropna().First_Name[emp.First_Name.dropna().str.contains(r'\d', regex=True)]

Series([], Name: First_Name, dtype: object)

In [57]:
emp.dropna().First_Name[emp.First_Name.dropna().str.contains(r'\W', regex=True)]

57    ?
Name: First_Name, dtype: object

? should be treated as missing for Name as well

### Lets read file again , supplying list of missing values 

In [67]:
emp = pd.read_csv(r'./data/employees_data.csv', na_values = ['?', 'na', 'n.a.', ])

In [68]:
emp.columns = emp.columns.str.replace(r'\s','_').str.replace(r'\W','').str.replace(r'_$','')

In [69]:
emp.head()

,First_Name,Gender,Salary,Bonus,Senior_Management,Team
0,Douglas,Male,97308.0,6.945,True,Marketing
1,Thomas,Male,61933.0,NaN,True,NaN
2,Maria,Female,130590.0,11.858,False,Finance
3,Jerry,Male,NaN,9.340,True,Finance
4,Larry,Male,101004.0,1.389,True,Client Services


In [70]:
emp.dtypes

First_Name            object
Gender                object
Salary               float64
Bonus                float64
Senior_Management     object
Team                  object
dtype: object

In [79]:
# infer Gender based on First Name

miss_gender_index = emp.Gender[emp.Gender.isna()].index

emp.iloc[miss_gender_index, [0,1]]

,First_Name,Gender
5,Dennis,NaN
8,Angela,NaN
15,Lillian,NaN
20,Lois,NaN
22,Joshua,NaN
...,...,...
961,Antonio,NaN
972,Victor,NaN
985,Stephen,NaN
989,Justin,NaN


In [81]:
# emp.Salary

emp[emp.Salary.isna()]

,First_Name,Gender,Salary,Bonus,Senior_Management,Team
3,Jerry,Male,NaN,9.340,True,Finance
8,Angela,NaN,NaN,18.523,True,Engineering
319,Jacqueline,Female,NaN,18.243,False,Marketing
349,Phyllis,Female,NaN,8.723,False,Sales
427,Frank,Male,NaN,8.037,True,Sales


In [85]:
# Based on Team, Senior_management, Bonus  try to infer  Salary

emp.query("Team == 'Finance' &  Senior_Management == True & Bonus >9 & Bonus < 10")

,First_Name,Gender,Salary,Bonus,Senior_Management,Team
3,Jerry,Male,NaN,9.340,True,Finance
67,Rachel,Female,51178.0,9.735,True,Finance
226,Kathy,Female,50905.0,9.404,True,Finance
